# Testing auto-differentiation of 'predict' method with Zygote.jl

### Model training

In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl`

In [2]:
using Flux
using Flux: gradient
using LaplaceRedux
using LinearAlgebra
using Plots
using Statistics
using Zygote

In [3]:
xs, ys = LaplaceRedux.Data.toy_data_non_linear(200)
X = hcat(xs...) # bring into tabular format
data = zip(xs,ys)

zip([[4.338278116518445, 4.346693861446744], [2.8895835101181366, 1.7724345503535346], [3.739171091869821, 4.806615763634292], [4.337448020194199, 2.8944493856125795], [4.385091999501366, 2.5993427745986546], [2.7179316296161153, 2.8763590202009115], [3.5510805948447106, 1.6071687255427378], [1.4107487578626614, 4.983832408881889], [2.3605816307989596, 0.8674613631761962], [1.1013194117557412, 4.54187896317418]  …  [-2.550765116941494, 0.7740255640038612], [-0.5689853766305939, 1.3681924968295949], [-3.0460870747317834, 1.4464977379569337], [-1.063558386399754, 0.9274420027171966], [-4.4029158762727105, 3.848201539780249], [-0.6533137738238128, 4.233217775766718], [-3.6867505640722884, 1.6399698341521611], [-4.115231059170212, 3.76570439406163], [-2.578551728943534, 3.09341159804621], [-1.127438275921857, 4.951438444293736]], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [4]:
n_hidden = 10
D = size(X,1)
nn = Chain(
    Dense(D, n_hidden, σ),
    Dense(n_hidden, 1)
)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y) 

2


Params([

Float32[0.62904763 -0.70134413; -0.23880176 0.59262556; 0.3062421 0.5838169; 0.41295835 0.3022863; -0.006728744 -0.48023972; 0.19677833 -0.15735787; -0.43431535 0.13617066; -0.37613237 -0.6355162; -0.5772489 0.5006516; 0.50735825 -0.57346106], 

Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[-0.47486153 0.24279788 0.027775105 -0.2827202 0.54958683 -0.32948235 0.19193204 0.18896899 0.1997307 0.33355826], Float32[0.0]])

loss (generic function with 1 method)

In [5]:
using Flux.Optimise: update!, Adam
opt = Adam(1e-3)
epochs = 100
avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(2 => 10, σ)
│   summary(x) = 2-element Vector{Float64}
└ @ Flux C:\Users\adeli\.julia\packages\Flux\FWgS0\src\layers\stateless.jl:50


Epoch 10


avg_loss(data) = 0.6624461008608341


Epoch 20
avg_loss(data) = 0.5739905554056167


Epoch 30
avg_loss(data) = 0.4420832905173302


...

Epoch 60
avg_loss(data) = 0.17378534894436598


Epoch 70
avg_loss(data) = 0.13618229190818965


Epoch 80
avg_loss(data) = 0.11063698114827275


Epoch 90
avg_loss(data) = 0.09237604660913348


Epoch 100
avg_loss(data) = 0.07880439629778266


In [6]:
la = Laplace(nn; likelihood=:classification, subset_of_weights=:all)
fit!(la, data)

200

### Auto-differentiation testing

In [8]:
# Define the function to be differentiated:
f(x) = predict(la, x)
# Differentiate it
J = jacobian(f, X)
println(J)

MethodError: MethodError: no method matching transform_jacobians(::LaplaceRedux.Curvature.EmpiricalFisher, ::Tuple{})
Closest candidates are:
  transform_jacobians(::LaplaceRedux.Curvature.CurvatureInterface, !Matched::Zygote.Grads) at c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\src\curvature\functions.jl:15

Through testing it has determined that the issue arises in nested use of Zygote, the use of `jacobian` in `jacobians` causes the issue. Jacobian moditifies the input arguments and therefore cannot be auto differentiated by Zygote. The [issue](https://github.com/FluxML/Zygote.jl/issues/953) of nested use of Zygote is still open.

Cloning the inpute does not solve the issue, since the Zygote operates on the principle of reducing the function to the rules it can differentiate.

Potential solutions:
- Use of ForwardDiff.jl or Tracker.jl instead of Zygote.jl